<a href="https://colab.research.google.com/github/R3M1d4Cod3r/AW1/blob/master/backbone_exercise_3_grl__starter_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**SCHELETRO PER IL PROGETTO DI ANOMALY SEGMENTATION**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)


**DOWNLOAD DATASET & SETUP THE ENVIRONMENT**

In [ ]:
# Download PACS Dataset Images
!git clone https://github.com/MachineLearning2020/Homework3-PACS/
!mv Homework3-PACS/PACS/ .
!rm -r Homework3-PACS/

# Download PACS Dataset Labels
!git clone https://github.com/silvia1993/DANN_Template/
!mv DANN_Template/txt_lists/art_painting.txt PACS/
!mv DANN_Template/txt_lists/cartoon.txt PACS/
!mv DANN_Template/txt_lists/photo.txt PACS/
!mv DANN_Template/txt_lists/sketch.txt PACS/
!rm -r DANN_Template/

# Install additional libraries
!pip install torchmetrics

Cloning into 'Homework3-PACS'...
remote: Enumerating objects: 10032, done.
remote: Total 10032 (delta 0), reused 0 (delta 0), pack-reused 10032 (from 1)
Receiving objects: 100% (10032/10032), 174.13 MiB | 31.67 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (9993/9993), done.
Cloning into 'DANN_Template'...
remote: Enumerating objects: 23, done.
remote: Total 23 (delta 0), reused 0 (delta 0), pack-reused 23 (from 1)
Receiving objects: 100% (23/23), 33.86 KiB | 1.47 MiB/s, done.
Resolving deltas: 100% (5/5), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 1.9 MB/s eta 0:00:00


**GLOBAL VARIABLES**

In [ ]:
import torch

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
NUM_CLASSES = 7
BATCH_SIZE = 256
LR = 1e-3            # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default
NUM_EPOCHS = 30      # Total number of training epochs (iterations over dataset)
STEP_SIZE = 20       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

**DATASET MANAGEMENT**

In [ ]:
from torch.utils.data import Dataset
import os

from PIL import Image
import sys


def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(f"/content/PACS/{path}", 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')


# Define the Dataset class
class PACSDataset(Dataset):
    def __init__(self, root= "/content/PACS/", source_domain="cartoon" , dataset_transform=None):
      super(PACSDataset, self).__init__( )

      self.dataset_transform = dataset_transform

      self.root=root
      # root= /content/PACS/
      self.data=[]
      self.labels = []
      path = self.root.split("/")[0]

      category_list = os.listdir(f"{root}/{source_domain}/")
      self.category = {}
      for i, e in enumerate(category_list):
        self.category[e] = i

      with open(f"{root}/{source_domain}.txt") as file:
        lines = file.readlines()

        for line in lines:
          #print(line)
          path, label = line.split()
          self.data.append(path.strip())
          self.labels.append(int(label))

      #print(self)


    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):

      image, label = pil_loader(self.data[index]), self.labels[index]

      # Applies preprocessing when accessing the image
      if self.dataset_transform is not None:
          image = self.dataset_transform(image)

      return image, label






**ARCHITECTURE SETUP**

In [ ]:
import torch.nn as nn

# Define AlexNet architecture class
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        # Category classifier
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )
        # Domain classifier
        self.domain_classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 2)
        )

    def forward(self, x):
        features = self.features(x)
        features = features.view(features.size(0), -1)
        class_outputs = self.classifier(features)
        domain_outputs = self.domain_classifier(features)
        return class_outputs, domain_outputs

**OPTIMIZATION LOOP**

In [ ]:
from torch.utils.data import DataLoader
import torchvision.transforms as T
from torchvision.models import AlexNet_Weights
import torch.nn.functional as F
from torchmetrics import Accuracy

#### DATA SETUP
# Define the transforms to use on images
dataset_transform = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define the Dataset object for training & testing
train_dataset = PACSDataset(source_domain="cartoon", dataset_transform=dataset_transform)
test_dataset = PACSDataset(source_domain="sketch", dataset_transform=dataset_transform)

# Define the DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)

LAMBDA = 0.005
#### ARCHITECTURE SETUP
# Create the Network Architecture object
model = AlexNet()
# Load pre-trained weights
model.load_state_dict(AlexNet_Weights.IMAGENET1K_V1.get_state_dict(progress=True), strict = False)
# Overwrite the final classifier layer as we only have 7 classes in PACS
model.classifier[-1] = nn.Linear(4096, 7)


#### TRAINING SETUP
# Move model to device before passing it to the optimizer
model = model.to(DEVICE)

parameters_to_optimize = model.parameters() # In this case we optimize over all the parameters of AlexNet
# Create Optimizer & Scheduler objects
optimizer = torch.optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)


#### TRAINING LOOP

#cudnn.benchmark # Calling this optimizes runtime
criterion = nn.CrossEntropyLoss()
current_step = 0
# Start iterating over the epochs
for epoch in range(NUM_EPOCHS):
  print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))

  model.train() # Sets module in training mode
  # Iterate over the dataset
  for (images_train, labels_train ), (images_test, labels_test) in zip(train_loader, test_loader):
    # Bring data over the device of choice
    images_train = images_train.to(DEVICE)
    labels_train = labels_train.to(DEVICE)
    images_test = images_test.to(DEVICE)
    labels_test = labels_test.to(DEVICE)



    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad() # Zero-ing the gradients

    # Forward pass to the network

    class_train , domain_train= model(images_train)
    class_test , domain_test = model(images_test)
    #print(class_train.dtype, class_train.shape)
    #print(labels_train.dtype, labels_train.shape)
    l1 = criterion(class_train, labels_train )
    l2 = criterion(domain_train, torch.zeros( BATCH_SIZE, dtype = torch.long , device = DEVICE) )
    l3 = criterion(domain_test, torch.ones(BATCH_SIZE,  dtype = torch.long , device = DEVICE) )

    loss = l1 + LAMBDA * (l2 + l3)
    # Compute loss based on output and ground truth


    # Log loss
    if current_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(current_step, loss.item()))

    # Compute gradients for each layer and update weights
    loss.backward()  # backward pass: computes gradients
    optimizer.step() # update weights based on accumulated gradients

    current_step += 1

  # Step the scheduler
  scheduler.step()

#### TEST LOOP
#...

#print(f'\nAccuracy on the target domain: {100 * accuracy:.2f}%')


Starting epoch 1/30, LR = [0.001]
Step 0, Loss 2.240873098373413
Starting epoch 2/30, LR = [0.001]
Step 10, Loss 0.7778012752532959
Starting epoch 3/30, LR = [0.001]
Step 20, Loss 0.5549272894859314
Starting epoch 4/30, LR = [0.001]
Step 30, Loss 0.39965537190437317
Starting epoch 5/30, LR = [0.001]
Step 40, Loss 0.33125758171081543
Starting epoch 6/30, LR = [0.001]
Step 50, Loss 0.22160929441452026
Starting epoch 7/30, LR = [0.001]
Step 60, Loss 0.21189571917057037
Starting epoch 8/30, LR = [0.001]
Step 70, Loss 0.21226322650909424
Starting epoch 9/30, LR = [0.001]
Step 80, Loss 0.19505062699317932
Starting epoch 10/30, LR = [0.001]
Starting epoch 11/30, LR = [0.001]
Step 90, Loss 0.1919592022895813
Starting epoch 12/30, LR = [0.001]
Step 100, Loss 0.11975648254156113
Starting epoch 13/30, LR = [0.001]
Step 110, Loss 0.10888698697090149
Starting epoch 14/30, LR = [0.001]
Step 120, Loss 0.14225244522094727
Starting epoch 15/30, LR = [0.001]
Step 130, Loss 0.11249377578496933
Starting e

NameError: name 'accuracy' is not defined

In [ ]:
model.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
model.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in test_loader:
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  # Forward Pass
  outputs = model(images)

  # Get predictions
  _, preds = torch.max(outputs[0], 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels).item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset)) * 100

print('Test Accuracy: {}'.format(accuracy))

Test Accuracy: 45.94044286077882
